In [1]:
import pandas as pd
import requests
from io import BytesIO
import warnings
import os 

### I noticed there was a difference in size of the parquet and csv files for the Green Jan 2019 Taxi data used in the 2023 cohort homework 1. So I wanted to check the other files that we are working with.    

In [ ]:
parquet_files = {
    "2021_01_yellow":"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet",
    "2019_01_green":"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-01.parquet",
    "2019_09_green":"https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2019-09.parquet"   
    }
csv_files = {
    "2021_01_yellow":"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2021-01.csv.gz",
    "2019_01_green":"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-01.csv.gz",
    "2019_09_green":"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-09.csv.gz"   
    }


In [ ]:
parquet_dfs = {}
csv_dfs = {}

# Suppress warnings temporarily
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for name, url in parquet_files.items():
        r = requests.get(url)
        #df = pd.read_parquet(BytesIO(r.content))
        parquet_dfs[name] = pd.read_parquet(BytesIO(r.content))
        print(name, parquet_dfs[name].shape)

    for name, url in csv_files.items():
        r = requests.get(url)
        #df = pd.read_csv(BytesIO(r.content), compression='gzip')
        csv_dfs[name] = pd.read_csv(BytesIO(r.content), compression='gzip')
        print(name, csv_dfs[name].shape)

In [ ]:
# TEST TO SEE IF THE DFs ARE EQUAL 
for name in parquet_dfs.keys():
    df_csv = csv_dfs[name].reindex(sorted(csv_dfs[name].columns), axis=1)
    df_parquet = parquet_dfs[name].reindex(sorted(parquet_dfs[name].columns), axis=1)
    are_equal = df_csv.equals(df_parquet)
    df_csv_means = df_csv.describe().loc['mean']
    df_parquet_means = df_parquet.describe().loc['mean']
    are_equal_means = df_csv_means.equals(df_parquet_means)
    print("DF Comp", name, are_equal)
    print("Column Means Comp", name, are_equal_means)



In [ ]:
## DOESNT WORK YET
    # Extract unique column headers
    # Extract unique column headers from both DataFrames
    unique_columns_csv = set(df_csv.columns.tolist())
    unique_columns_parquet = set(df_parquet.columns.tolist())

    # Combine unique columns from both DataFrames
    unique_columns_combined = unique_columns_csv.union(unique_columns_parquet)

    # Convert the combined unique columns to a list
    unique_columns_combined = list(unique_columns_combined)

    # Create an empty DataFrame with the unique column headers as columns
    combined_df = pd.DataFrame(columns=unique_columns_combined)

    # Populate the combined DataFrame with values from df_csv
    for column in df_csv.columns:
        combined_df[column] = df_csv_means[column]

    # Populate the combined DataFrame with values from df_parquet
    for column in df_parquet.columns:
        combined_df[column] = df_parquet_means[column]

    # Now combined_df contains all unique column headers and their respective values
    print(combined_df)